## Writing this program in ipython notebook, since it is easy to visualize the results and stop after a step with data in memory..

### Importing necessary modules

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup

## URL for indian medical register

In [2]:
url = "https://www.nmc.org.in/information-desk/indian-medical-register/"

### Starting the driver

In [3]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(url)
driver.maximize_window()

### Getting the table containing doctors name and data

In [4]:
time.sleep(5)  # time to load the page
select_year = driver.find_element(By.ID, "doctorYear")
select = Select(select_year)

In [5]:
year = driver.find_element(By.XPATH, '//*[@id="doctorYear"]/option[4]')
driver.execute_script('arguments[0].setAttribute("style","display:block;");', select_year)

In [6]:
select.select_by_visible_text('2020')

In [8]:
# Selecting state
select_state = driver.find_element(By.ID, "advsmcId")
select = Select(select_state)
bihar_state = driver.find_element(By.XPATH, '//*[@id="advsmcId"]/option[6]')
driver.execute_script('arguments[0].setAttribute("style","display:block;");', select_state)
select.select_by_value('4')

In [9]:
submit = driver.find_element(By.ID, "doctor_advance_Details")
submit.click()
time.sleep(5)

# Get web page

In [17]:
from io import StringIO
import pandas as pd
all_doctors = []
while True:
    webpage = driver.page_source
    df = pd.read_html(StringIO(webpage))
    all_doctors.append(df[-1])
    
    next_button = driver.find_element(By.ID, "doct_info5_next")
    classes = next_button.get_attribute('class')
    if 'disabled' in classes:
        break
    else:
        next_button.click()
        time.sleep(5)

In [19]:
all_doctors[0]

,Sl. No.,Year of Info,Registration Number,State Medical Councils,Name,Father Name,Action
0,1,2020,49211,Bihar Medical Council,Md.Shabbir Alam,S/o Md.Aayub Ali,View
1,2,2020,49212,Bihar Medical Council,KM Kaneez Fatma,D/o Mrs Abu Zafar,View
2,3,2020,49213,Bihar Medical Council,Sabina Yasmin,D/o Abul Hussain,View
3,4,2020,49214,Bihar Medical Council,Aaditya Jolly,D/o Dr. Brij Bihari Singh,View
4,5,2020,49215,Bihar Medical Council,Saher Fatma`,D/o Afi Siddiqui,View
...,...,...,...,...,...,...,...
495,496,2020,49725,Bihar Medical Council,Vishwajeet Kumar,S/o Surendra Singh,View
496,497,2020,49726,Bihar Medical Council,Sanjeeda ali,D/o Md yusuf,View
497,498,2020,49727,Bihar Medical Council,Rishu Kumar,S/o Kamlesh Kumar Singh,View
498,499,2020,49728,Bihar Medical Council,Hemant Kumar Suman,S/o Hari Narayan Paswan,View


In [20]:
# combining all_doctors
all_doctors_df = pd.concat(all_doctors, axis=0)
all_doctors_df

,Sl. No.,Year of Info,Registration Number,State Medical Councils,Name,Father Name,Action
0,1,2020,49211,Bihar Medical Council,Md.Shabbir Alam,S/o Md.Aayub Ali,View
1,2,2020,49212,Bihar Medical Council,KM Kaneez Fatma,D/o Mrs Abu Zafar,View
2,3,2020,49213,Bihar Medical Council,Sabina Yasmin,D/o Abul Hussain,View
3,4,2020,49214,Bihar Medical Council,Aaditya Jolly,D/o Dr. Brij Bihari Singh,View
4,5,2020,49215,Bihar Medical Council,Saher Fatma`,D/o Afi Siddiqui,View
...,...,...,...,...,...,...,...
456,1457,2020,50725,Bihar Medical Council,Vibhash Kumar,S/o Angad Sharma,View
457,1458,2020,50726,Bihar Medical Council,Subodh Kumar Singh,S/o Badri Singh,View
458,1459,2020,50727,Bihar Medical Council,Depak Kumar Vishwakarma,S/o Jagdish Vishwakarma,View
459,1460,2020,50730,Bihar Medical Council,Shivang Bharti,S/O Om Prakash Gupta,View


In [23]:
all_doctors_df.columns

Index(['Sl. No.', 'Year of Info', 'Registration Number',
       'State Medical Councils', 'Name', 'Father Name', 'Action'],
      dtype='object')

In [24]:
all_doctors_df.to_csv('all_doctors.csv', index=False, columns=['Sl. No.', 'Year of Info', 'Registration Number', 'State Medical Councils', 'Name', 'Father Name'])

In [ ]:
driver.quit()